# Ugesco : enrich and desambiguate a sample of the Samnang-Hans' JSON

In [1]:
from ugesco import *

D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, fa

In [2]:
#test sur un Json contenant 20 éléments pris au hasard
with open(r"C:\Users\ettor\Documents\GitHub\UGESCO\data\jsondata_ugesco.json", "r", encoding="utf8") as file:
    json = json.load(file)
    
sample = random.sample(json, k=20)


In [3]:
data = pd.DataFrame([flatten(line) for line in sample])

In [4]:
data.head()

,beeldid,imageclassification_class1,imageclassification_class2,imageclassification_class3,imageclassification_class4,imageclassification_class5,imageclassification_prob1,imageclassification_prob2,imageclassification_prob3,imageclassification_prob4,...,spatial_key_3,spatial_value_0,spatial_value_1,spatial_value_2,spatial_value_3,temporal,temporal_key_0,temporal_key_1,temporal_value_0,temporal_value_1
0,00017136,plaza,apartment_building/outdoor,hotel/outdoor,office_building,street,0.402744,0.219915,0.059944,0.046442,...,NaN,Anvers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00000146,street,ski_resort,building_facade,residential_neighborhood,fire_escape,0.193325,0.184395,0.147117,0.083031,...,NaN,Namur,NaN,NaN,NaN,NaN,TIMEX_DATE,NaN,6 juin 1940,NaN
2,00005572,crosswalk,street,plaza,landing_deck,army_base,0.907207,0.055784,0.008323,0.006928,...,NaN,Bruxelles,Boulevard Anspacht,Galeries Anspach,NaN,NaN,TIMEX_DATE,NaN,23/8/1944,NaN
3,00000909,plaza,street,promenade,downtown,tower,0.505309,0.120646,0.084693,0.059608,...,NaN,Ostende,NaN,NaN,NaN,NaN,TIMEX_DATE,NaN,Mai 1940,NaN
4,00005535,crosswalk,street,alley,plaza,fire_escape,0.614272,0.373704,0.005196,0.001948,...,NaN,Bruxelles,Bruxelles,NaN,NaN,NaN,TIMEX_DURATION,NaN,pendant la guerre,NaN


In [5]:
# exclude image classifications and temporal values for the moment
data = data[data.columns.drop(list(data.filter(regex='imageclassification|temporal')))]

In [6]:
# reshape the dataframe verticaly
data = data.set_index('beeldid')
data.columns = data.columns.str.split('_', expand=True)

In [7]:
data = data.stack().reset_index(level=1, drop=True).reset_index()

In [8]:
data.head()

beeldid       spatial                    
                      key               value
0  00017136     LOC_PLACE              Anvers
1  00000146     LOC_PLACE               Namur
2  00005572     LOC_PLACE           Bruxelles
3  00005572    LOC_STREET  Boulevard Anspacht
4  00005572  LOC_BUILDING    Galeries Anspach

In [9]:
data.columns = ['_'.join(col) for col in data.columns]

In [10]:
data.rename(columns={'beeldid_': 'beeldid'}, inplace=True)

In [11]:
data.head()

,beeldid,spatial_key,spatial_value
0,00017136,LOC_PLACE,Anvers
1,00000146,LOC_PLACE,Namur
2,00005572,LOC_PLACE,Bruxelles
3,00005572,LOC_STREET,Boulevard Anspacht
4,00005572,LOC_BUILDING,Galeries Anspach


In [12]:
#join/merge with phototheque_pallas to get the locations. 
#The csv is gzipped to get around the 100MB file weigth limitation of Github
phototeque = pd.read_csv("data/phototheque_pallas.csv.gz", compression="gzip", encoding="utf8", dtype={'beeldid': str, 'trfwnumm': str})

data = pd.merge(data, phototeque, how='left', on=['beeldid'], suffixes=['', '_x'])

In [13]:
# Pictures that contains more than a loc in the thesaurus descriptors

#data[data['loc_qid'].str.contains(",", na = False)]

In [14]:
data.head()

,beeldid,spatial_key,spatial_value,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name
0,00017136,LOC_PLACE,Anvers,http://pallas.cegesoma.be/pls/opac/opac.search...,De eerste mars ging door de stad Antwerpen. Hi...,Semaine culturelle organisée par la Jeunesse n...,749,NaN,Q31,BE,Belgique,Q12892,Anvers
1,00000146,LOC_PLACE,Namur,http://pallas.cegesoma.be/pls/opac/opac.search...,Coin rue L. Namèche et rue Rogier à Namur (6 j...,"Bombardement de Namur par les Allemands, 5/194...",9,campagne des 18 jours (1940)--namur (belgique),Q31,BE,Belgique,Q134121,Namur
2,00005572,LOC_PLACE,Bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,La chaleur à Bruxelles le 23/8/1944. Boulevard...,"Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles
3,00005572,LOC_STREET,Boulevard Anspacht,http://pallas.cegesoma.be/pls/opac/opac.search...,La chaleur à Bruxelles le 23/8/1944. Boulevard...,"Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles
4,00005572,LOC_BUILDING,Galeries Anspach,http://pallas.cegesoma.be/pls/opac/opac.search...,La chaleur à Bruxelles le 23/8/1944. Boulevard...,"Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles


In [15]:
# Merge with NER_classes (spatial_keys matched with wikidata)
ner_classes = pd.read_csv("data/ner_classes.csv")

data = pd.merge(data, ner_classes, how='left', left_on = ['spatial_key'], right_on = ['ner_class'])

In [16]:
data.head()

,beeldid,spatial_key,spatial_value,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class,wiki_qid,wiki_class
0,00017136,LOC_PLACE,Anvers,http://pallas.cegesoma.be/pls/opac/opac.search...,De eerste mars ging door de stad Antwerpen. Hi...,Semaine culturelle organisée par la Jeunesse n...,749,NaN,Q31,BE,Belgique,Q12892,Anvers,LOC_PLACE,Q56061,administrative territorial entity
1,00000146,LOC_PLACE,Namur,http://pallas.cegesoma.be/pls/opac/opac.search...,Coin rue L. Namèche et rue Rogier à Namur (6 j...,"Bombardement de Namur par les Allemands, 5/194...",9,campagne des 18 jours (1940)--namur (belgique),Q31,BE,Belgique,Q134121,Namur,LOC_PLACE,Q56061,administrative territorial entity
2,00005572,LOC_PLACE,Bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,La chaleur à Bruxelles le 23/8/1944. Boulevard...,"Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,LOC_PLACE,Q56061,administrative territorial entity
3,00005572,LOC_STREET,Boulevard Anspacht,http://pallas.cegesoma.be/pls/opac/opac.search...,La chaleur à Bruxelles le 23/8/1944. Boulevard...,"Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,LOC_STREET,Q83620,thoroughfare
4,00005572,LOC_BUILDING,Galeries Anspach,http://pallas.cegesoma.be/pls/opac/opac.search...,La chaleur à Bruxelles le 23/8/1944. Boulevard...,"Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,LOC_BUILDING,Q41176,building


In [17]:
# rename the new columns merged
data.rename(columns={'wiki_qid': 'ner_class_qid', 'wiki_class':'ner_class_name'}, inplace=True)

In [18]:
#remove duplicate columns
data = data.T.drop_duplicates().T

In [19]:
data.head()

,beeldid,spatial_key,spatial_value,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00017136,LOC_PLACE,Anvers,http://pallas.cegesoma.be/pls/opac/opac.search...,De eerste mars ging door de stad Antwerpen. Hi...,Semaine culturelle organisée par la Jeunesse n...,749,NaN,Q31,BE,Belgique,Q12892,Anvers,Q56061,administrative territorial entity
1,00000146,LOC_PLACE,Namur,http://pallas.cegesoma.be/pls/opac/opac.search...,Coin rue L. Namèche et rue Rogier à Namur (6 j...,"Bombardement de Namur par les Allemands, 5/194...",9,campagne des 18 jours (1940)--namur (belgique),Q31,BE,Belgique,Q134121,Namur,Q56061,administrative territorial entity
2,00005572,LOC_PLACE,Bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,La chaleur à Bruxelles le 23/8/1944. Boulevard...,"Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
3,00005572,LOC_STREET,Boulevard Anspacht,http://pallas.cegesoma.be/pls/opac/opac.search...,La chaleur à Bruxelles le 23/8/1944. Boulevard...,"Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q83620,thoroughfare
4,00005572,LOC_BUILDING,Galeries Anspach,http://pallas.cegesoma.be/pls/opac/opac.search...,La chaleur à Bruxelles le 23/8/1944. Boulevard...,"Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q41176,building


In [20]:
# remove articles in spatial_value
data['spatial_value'] = data['spatial_value'].str.replace(pat=r"\b(le|la|l\s+'|l'|les)\b", repl="", n=1, case=False)

In [21]:
#data.to_csv(r"C:\Users\ettor\Desktop\data_ugesco.csv", header=True, index=False, encoding="utf-8")

In [22]:
data.head()

,beeldid,spatial_key,spatial_value,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00017136,LOC_PLACE,Anvers,http://pallas.cegesoma.be/pls/opac/opac.search...,De eerste mars ging door de stad Antwerpen. Hi...,Semaine culturelle organisée par la Jeunesse n...,749,NaN,Q31,BE,Belgique,Q12892,Anvers,Q56061,administrative territorial entity
1,00000146,LOC_PLACE,Namur,http://pallas.cegesoma.be/pls/opac/opac.search...,Coin rue L. Namèche et rue Rogier à Namur (6 j...,"Bombardement de Namur par les Allemands, 5/194...",9,campagne des 18 jours (1940)--namur (belgique),Q31,BE,Belgique,Q134121,Namur,Q56061,administrative territorial entity
2,00005572,LOC_PLACE,Bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,La chaleur à Bruxelles le 23/8/1944. Boulevard...,"Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
3,00005572,LOC_STREET,Boulevard Anspacht,http://pallas.cegesoma.be/pls/opac/opac.search...,La chaleur à Bruxelles le 23/8/1944. Boulevard...,"Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q83620,thoroughfare
4,00005572,LOC_BUILDING,Galeries Anspach,http://pallas.cegesoma.be/pls/opac/opac.search...,La chaleur à Bruxelles le 23/8/1944. Boulevard...,"Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q41176,building


In [23]:
#normalize spatial_value
data['spatial_value'] = data['spatial_value'].apply(lambda x: Fingerprinter(x).get_fingerprint_nonsorted())

In [24]:
#transformer les loc_place pays en loc_country
with open(r"C:\Users\ettor\Documents\GitHub\UGESCO\data\countries.csv", 'r', encoding="utf8") as f:
    countries = set([Fingerprinter(country).get_fingerprint_nonsorted() for country in f])

# Grab DataFrame rows where column value is present in a list
data.loc[(data['spatial_value'].isin(countries)), ['spatial_key']] = "LOC_COUNTRY"

In [25]:
data

,beeldid,spatial_key,spatial_value,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00017136,LOC_PLACE,anvers,http://pallas.cegesoma.be/pls/opac/opac.search...,De eerste mars ging door de stad Antwerpen. Hi...,Semaine culturelle organisée par la Jeunesse n...,749,NaN,Q31,BE,Belgique,Q12892,Anvers,Q56061,administrative territorial entity
1,00000146,LOC_PLACE,namur,http://pallas.cegesoma.be/pls/opac/opac.search...,Coin rue L. Namèche et rue Rogier à Namur (6 j...,"Bombardement de Namur par les Allemands, 5/194...",9,campagne des 18 jours (1940)--namur (belgique),Q31,BE,Belgique,Q134121,Namur,Q56061,administrative territorial entity
2,00005572,LOC_PLACE,bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,La chaleur à Bruxelles le 23/8/1944. Boulevard...,"Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
3,00005572,LOC_STREET,boulevard anspacht,http://pallas.cegesoma.be/pls/opac/opac.search...,La chaleur à Bruxelles le 23/8/1944. Boulevard...,"Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q83620,thoroughfare
4,00005572,LOC_BUILDING,galeries anspach,http://pallas.cegesoma.be/pls/opac/opac.search...,La chaleur à Bruxelles le 23/8/1944. Boulevard...,"Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q41176,building
5,00000909,LOC_PLACE,ostende,http://pallas.cegesoma.be/pls/opac/opac.search...,Mai 1940: une grande partie de la population c...,"Réfugiés belges de l'exode, 5/1940. - 64 photos",80,NaN,Q142,FR,France,NaN,NaN,Q56061,administrative territorial entity
6,00005535,LOC_PLACE,bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles pendant la guerre: Le fiacre avait d...,"Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
7,00005535,LOC_PLACE,bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles pendant la guerre: Le fiacre avait d...,"Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
8,00040508,LOC_PLACE,bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,"La marche de la faim sur Bruxelles. La ""Marche...","Henri De Boek, membre communiste du conseil co...",3406,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
9,00040508,LOC_PLACE,bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,"La marche de la faim sur Bruxelles. La ""Marche...","Henri De Boek, membre communiste du conseil co...",3406,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity


In [ ]:
gb = data.groupby(['beeldid'])

In [26]:
gb.head() 

,beeldid,spatial_key,spatial_value,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00017136,LOC_PLACE,anvers,http://pallas.cegesoma.be/pls/opac/opac.search...,De eerste mars ging door de stad Antwerpen. Hi...,Semaine culturelle organisée par la Jeunesse n...,749,NaN,Q31,BE,Belgique,Q12892,Anvers,Q56061,administrative territorial entity
1,00000146,LOC_PLACE,namur,http://pallas.cegesoma.be/pls/opac/opac.search...,Coin rue L. Namèche et rue Rogier à Namur (6 j...,"Bombardement de Namur par les Allemands, 5/194...",9,campagne des 18 jours (1940)--namur (belgique),Q31,BE,Belgique,Q134121,Namur,Q56061,administrative territorial entity
2,00005572,LOC_PLACE,bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,La chaleur à Bruxelles le 23/8/1944. Boulevard...,"Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
3,00005572,LOC_STREET,boulevard anspacht,http://pallas.cegesoma.be/pls/opac/opac.search...,La chaleur à Bruxelles le 23/8/1944. Boulevard...,"Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q83620,thoroughfare
4,00005572,LOC_BUILDING,galeries anspach,http://pallas.cegesoma.be/pls/opac/opac.search...,La chaleur à Bruxelles le 23/8/1944. Boulevard...,"Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q41176,building
5,00000909,LOC_PLACE,ostende,http://pallas.cegesoma.be/pls/opac/opac.search...,Mai 1940: une grande partie de la population c...,"Réfugiés belges de l'exode, 5/1940. - 64 photos",80,NaN,Q142,FR,France,NaN,NaN,Q56061,administrative territorial entity
6,00005535,LOC_PLACE,bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles pendant la guerre: Le fiacre avait d...,"Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
7,00005535,LOC_PLACE,bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles pendant la guerre: Le fiacre avait d...,"Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
8,00040508,LOC_PLACE,bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,"La marche de la faim sur Bruxelles. La ""Marche...","Henri De Boek, membre communiste du conseil co...",3406,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
9,00040508,LOC_PLACE,bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,"La marche de la faim sur Bruxelles. La ""Marche...","Henri De Boek, membre communiste du conseil co...",3406,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity


In [ ]:
#Sélectionner les records qui contiennent moins de 3 loc_place (attention : brussels et bruxelles = 1 en fait. D'où, deux semble un bon compromis)
df_test = gb['spatial_key'].apply(lambda x: (x=='LOC_PLACE').sum()).reset_index(name='count_places')

In [27]:
df_test

,beeldid,count_places
0,00000146,1
1,00000429,0
2,00000909,1
3,00004445,1
4,00005492,0
5,00005535,2
6,00005547,0
7,00005572,1
8,00007170,1
9,00007202,2


In [28]:
filtre = list(df_test[df_test['count_places'] < 3]['beeldid'])  
gb_filtered = data[data['beeldid'].isin(filtre)].groupby('beeldid')

In [29]:
gb_filtered.head()

,beeldid,spatial_key,spatial_value,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00017136,LOC_PLACE,anvers,http://pallas.cegesoma.be/pls/opac/opac.search...,De eerste mars ging door de stad Antwerpen. Hi...,Semaine culturelle organisée par la Jeunesse n...,749,NaN,Q31,BE,Belgique,Q12892,Anvers,Q56061,administrative territorial entity
1,00000146,LOC_PLACE,namur,http://pallas.cegesoma.be/pls/opac/opac.search...,Coin rue L. Namèche et rue Rogier à Namur (6 j...,"Bombardement de Namur par les Allemands, 5/194...",9,campagne des 18 jours (1940)--namur (belgique),Q31,BE,Belgique,Q134121,Namur,Q56061,administrative territorial entity
2,00005572,LOC_PLACE,bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,La chaleur à Bruxelles le 23/8/1944. Boulevard...,"Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
3,00005572,LOC_STREET,boulevard anspacht,http://pallas.cegesoma.be/pls/opac/opac.search...,La chaleur à Bruxelles le 23/8/1944. Boulevard...,"Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q83620,thoroughfare
4,00005572,LOC_BUILDING,galeries anspach,http://pallas.cegesoma.be/pls/opac/opac.search...,La chaleur à Bruxelles le 23/8/1944. Boulevard...,"Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q41176,building
5,00000909,LOC_PLACE,ostende,http://pallas.cegesoma.be/pls/opac/opac.search...,Mai 1940: une grande partie de la population c...,"Réfugiés belges de l'exode, 5/1940. - 64 photos",80,NaN,Q142,FR,France,NaN,NaN,Q56061,administrative territorial entity
6,00005535,LOC_PLACE,bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles pendant la guerre: Le fiacre avait d...,"Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
7,00005535,LOC_PLACE,bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles pendant la guerre: Le fiacre avait d...,"Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
12,00007202,LOC_PLACE,bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,"L'actrice de cinéma allemande, Ilse Werner à B...","Ilse Werner, actrice de cinéma, 1942. - 3 photos",147847,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
13,00007202,LOC_STREET,grand'place,http://pallas.cegesoma.be/pls/opac/opac.search...,"L'actrice de cinéma allemande, Ilse Werner à B...","Ilse Werner, actrice de cinéma, 1942. - 3 photos",147847,NaN,NaN,NaN,NaN,NaN,NaN,Q83620,thoroughfare


In [ ]:
#et maintenant appliquer la concatenation uniquement si le record ne contient pas plus de deux loc_place
gb = pd.DataFrame(gb_filtered['spatial_value'].apply(lambda x: "%s" % ' '.join(x))).reset_index()

In [30]:
gb.head()

,beeldid,spatial_value
0,00000146,namur
1,00000429,belgique
2,00000909,ostende
3,00004445,grand'poste bruxelles
4,00005492,place de monnaie


In [ ]:
#comment concaténer loc_name maintenant, et accéder à country_name ?
#en attendant de trouver une solution élégante, récréer un dataset contenant ces infos

gb = gb.merge(data[["beeldid", "loc_name", "country_name"]])

In [31]:
gb.head()

,beeldid,spatial_value,loc_name,country_name
0,00000146,namur,Namur,Belgique
1,00000429,belgique,NaN,Belgique
2,00000909,ostende,NaN,France
3,00004445,grand'poste bruxelles,NaN,Belgique
4,00004445,grand'poste bruxelles,NaN,Belgique


In [32]:
#on splitte les noms de pays multiples (n'existent plus)
#gb = tidy_split(gb.merge(data[["beeldid", "country_name", "loc_name"]]), column="country_name", sep=",", keep=False)

In [33]:
#on enlève les doublons
gb.drop_duplicates(subset=None, keep='first', inplace=True)

In [34]:
gb

,beeldid,spatial_value,loc_name,country_name
0,00000146,namur,Namur,Belgique
1,00000429,belgique,NaN,Belgique
2,00000909,ostende,NaN,France
3,00004445,grand'poste bruxelles,NaN,Belgique
5,00005492,place de monnaie,Bruxelles,Belgique
6,00005535,bruxelles bruxelles,Bruxelles,Belgique
8,00005547,au petit sablon conservatoire,Bruxelles,Belgique
10,00005572,bruxelles boulevard anspacht galeries anspach,Bruxelles,Belgique
13,00007170,bruxelles allemagne,NaN,NaN
15,00007202,bruxelles grand'place bruxelles,NaN,NaN


In [35]:
#on normalise les lieux et pays
gb['loc_name'] = gb['loc_name'].apply(lambda x: Fingerprinter(str(x)).get_fingerprint_nonsorted()).str.replace('nan',  '')
gb['country_name'] = gb['country_name'].apply(lambda x: Fingerprinter(str(x)).get_fingerprint_nonsorted()).str.replace('nan',  '')

In [ ]:
# on concatène le tout en supprimant les doublons

gb['loc_concat'] = gb['spatial_value'].fillna('').map(
    str) + ' ' + gb['loc_name'].fillna('').map(str) + ' ' + gb['country_name'].fillna('').map(str)

gb['loc_concat'] = gb['loc_concat'].apply(
    lambda x: ' '.join(pd.unique(x.split())))

In [36]:
gb

,beeldid,spatial_value,loc_name,country_name,loc_concat
0,00000146,namur,namur,belgique,namur belgique
1,00000429,belgique,,belgique,belgique
2,00000909,ostende,,france,ostende france
3,00004445,grand'poste bruxelles,,belgique,grand'poste bruxelles belgique
5,00005492,place de monnaie,bruxelles,belgique,place de monnaie bruxelles belgique
6,00005535,bruxelles bruxelles,bruxelles,belgique,bruxelles belgique
8,00005547,au petit sablon conservatoire,bruxelles,belgique,au petit sablon conservatoire bruxelles belgique
10,00005572,bruxelles boulevard anspacht galeries anspach,bruxelles,belgique,bruxelles boulevard anspacht galeries anspach ...
13,00007170,bruxelles allemagne,,,bruxelles allemagne
15,00007202,bruxelles grand'place bruxelles,,,bruxelles grand'place


In [37]:
gb.iat[0,4]

'namur belgique'

In [38]:
#appliquer get_nominatim. ATTENTION : ajouter le pays

# Nominatim bloque les requêtes répétées
requests_cache.install_cache('nominatim_cache')

s = requests.Session()

def get_nominatim(value, countrycodes=['BE',''], limit=5, lang="fr"):
    # doc : https://wiki.openstreetmap.org/wiki/Nominatim
    url = 'http://nominatim.openstreetmap.org/'
    

    params = {'q': value,
              'format': 'jsonv2',
              'addressdetails': 1,
              'limit': limit,
              'email': 'ettorerizza@mail.com',
              'polygon_kml': 0,
              'extratags': 1,
              'namedetails': 0,
              # ajouter un country_code vide simule 
              # la péréférence pour un pays de geopy
              # sans se limiter à celui-ci
              'countrycodes': countrycodes,
              'accept-language': lang
              }

    headers = {
        'User-Agent': 'Ugesco app',
        'From': 'ettorerizza@mail.com'
    }

    result = s.get(url, params=params, headers=headers, timeout=10).json()

    # Nominatim bloque les requêtes trop rapides
    sleep(1)

    return result

gb['nominatim'] = gb['loc_concat'].apply(get_nominatim)

D:\Anaconda3\lib\socket.py:657: ResourceWarning: unclosed <socket.socket fd=2532, family=AddressFamily.AF_INET6, type=SocketKind.SOCK_STREAM, proto=0, laddr=('2a02:1811:3700:1500:9d5d:d43f:9e40:1c8d', 50799, 0, 0), raddr=('2001:630:12:500:ec4:7aff:fe66:96d2', 443, 0, 0)>
  self._sock = None
D:\Anaconda3\lib\socket.py:657: ResourceWarning: unclosed <socket.socket fd=2888, family=AddressFamily.AF_INET6, type=SocketKind.SOCK_STREAM, proto=0, laddr=('2a02:1811:3700:1500:9d5d:d43f:9e40:1c8d', 50802, 0, 0), raddr=('2001:630:12:500:ec4:7aff:fe66:96d2', 443, 0, 0)>
  self._sock = None
D:\Anaconda3\lib\socket.py:657: ResourceWarning: unclosed <socket.socket fd=2896, family=AddressFamily.AF_INET6, type=SocketKind.SOCK_STREAM, proto=0, laddr=('2a02:1811:3700:1500:9d5d:d43f:9e40:1c8d', 50804, 0, 0), raddr=('2001:630:12:500:ec4:7aff:fe66:96d2', 443, 0, 0)>
  self._sock = None
D:\Anaconda3\lib\socket.py:657: ResourceWarning: unclosed <socket.socket fd=2952, family=AddressFamily.AF_INET6, type=Socket

In [39]:
gb.head(10)

,beeldid,spatial_value,loc_name,country_name,loc_concat,nominatim
0,00000146,namur,namur,belgique,namur belgique,"[{'place_id': '178244718', 'licence': 'Data © ..."
1,00000429,belgique,,belgique,belgique,"[{'place_id': '179136572', 'licence': 'Data © ..."
2,00000909,ostende,,france,ostende france,"[{'place_id': '48801331', 'licence': 'Data © O..."
3,00004445,grand'poste bruxelles,,belgique,grand'poste bruxelles belgique,[]
5,00005492,place de monnaie,bruxelles,belgique,place de monnaie bruxelles belgique,"[{'place_id': '101462620', 'licence': 'Data © ..."
6,00005535,bruxelles bruxelles,bruxelles,belgique,bruxelles belgique,"[{'place_id': '17872759', 'licence': 'Data © O..."
8,00005547,au petit sablon conservatoire,bruxelles,belgique,au petit sablon conservatoire bruxelles belgique,[]
10,00005572,bruxelles boulevard anspacht galeries anspach,bruxelles,belgique,bruxelles boulevard anspacht galeries anspach ...,[]
13,00007170,bruxelles allemagne,,,bruxelles allemagne,[]
15,00007202,bruxelles grand'place bruxelles,,,bruxelles grand'place,"[{'place_id': '62161128', 'licence': 'Data © O..."


In [ ]:
#Comment attribuer un score de fiabilité aux résultats et choisir le meilleur s'il y en a un ?

def get_element(array, *args):
    """parse the first nominatim json."""
    for el in array:
        return dig(el, *args)
    
gb['main_loc'] = np.vectorize(get_element)(gb['nominatim'], "display_name")
gb['type_loc'] = np.vectorize(get_element)(gb['nominatim'], "type")
gb['city_loc'] = np.vectorize(get_element)(gb['nominatim'], "address", "city")
gb['country_loc'] = np.vectorize(get_element)(gb['nominatim'], "address", "country") 

In [40]:
gb 

,beeldid,spatial_value,loc_name,country_name,loc_concat,nominatim,main_loc,type_loc,city_loc,country_loc
0,00000146,namur,namur,belgique,namur belgique,"[{'place_id': '178244718', 'licence': 'Data © ...","Namur, Wallonie, Belgique",city,Namur,Belgique
1,00000429,belgique,,belgique,belgique,"[{'place_id': '179136572', 'licence': 'Data © ...",Belgique,administrative,None,Belgique
2,00000909,ostende,,france,ostende france,"[{'place_id': '48801331', 'licence': 'Data © O...","Ostende, Rue d'Ostende, Saint-Quentin, Aisne, ...",bus_stop,None,France
3,00004445,grand'poste bruxelles,,belgique,grand'poste bruxelles belgique,[],None,None,None,None
5,00005492,place de monnaie,bruxelles,belgique,place de monnaie bruxelles belgique,"[{'place_id': '101462620', 'licence': 'Data © ...","Place de la Monnaie, Quartier du Centre, Bruxe...",pedestrian,Ville de Bruxelles,Belgique
6,00005535,bruxelles bruxelles,bruxelles,belgique,bruxelles belgique,"[{'place_id': '17872759', 'licence': 'Data © O...","Bruxelles, Bruxelles-Capitale, 1000, Belgique",city,Bruxelles,Belgique
8,00005547,au petit sablon conservatoire,bruxelles,belgique,au petit sablon conservatoire bruxelles belgique,[],None,None,None,None
10,00005572,bruxelles boulevard anspacht galeries anspach,bruxelles,belgique,bruxelles boulevard anspacht galeries anspach ...,[],None,None,None,None
13,00007170,bruxelles allemagne,,,bruxelles allemagne,[],None,None,None,None
15,00007202,bruxelles grand'place bruxelles,,,bruxelles grand'place,"[{'place_id': '62161128', 'licence': 'Data © O...","Grand-Place, Quartier du Centre, Bruxelles, Vi...",pedestrian,Ville de Bruxelles,Belgique


In [41]:
#en attendant de trouver un moyen de chosir le bon Nominatim, traitons avec Wikipedia les beeldid dont main_loc est None
import wikipedia
def get_wikipedia(value):
    wikipedia.set_lang("fr")
    if value is not None:
        try:
            return wikipedia.search(value, results=1, suggestion=False)[0]
        except IndexError:
            return None


In [42]:
gb['wikipedia_test'] = np.vectorize(get_wikipedia)(gb['loc_concat']) 

In [43]:
gb

,beeldid,spatial_value,loc_name,country_name,loc_concat,nominatim,main_loc,type_loc,city_loc,country_loc,wikipedia_test
0,00000146,namur,namur,belgique,namur belgique,"[{'place_id': '178244718', 'licence': 'Data © ...","Namur, Wallonie, Belgique",city,Namur,Belgique,Namur
1,00000429,belgique,,belgique,belgique,"[{'place_id': '179136572', 'licence': 'Data © ...",Belgique,administrative,None,Belgique,Belgique
2,00000909,ostende,,france,ostende france,"[{'place_id': '48801331', 'licence': 'Data © O...","Ostende, Rue d'Ostende, Saint-Quentin, Aisne, ...",bus_stop,None,France,Ostende
3,00004445,grand'poste bruxelles,,belgique,grand'poste bruxelles belgique,[],None,None,None,None,Verviers
5,00005492,place de monnaie,bruxelles,belgique,place de monnaie bruxelles belgique,"[{'place_id': '101462620', 'licence': 'Data © ...","Place de la Monnaie, Quartier du Centre, Bruxe...",pedestrian,Ville de Bruxelles,Belgique,Place de la Monnaie (Bruxelles)
6,00005535,bruxelles bruxelles,bruxelles,belgique,bruxelles belgique,"[{'place_id': '17872759', 'licence': 'Data © O...","Bruxelles, Bruxelles-Capitale, 1000, Belgique",city,Bruxelles,Belgique,Communauté française de Belgique
8,00005547,au petit sablon conservatoire,bruxelles,belgique,au petit sablon conservatoire bruxelles belgique,[],None,None,None,None,Ville de Bruxelles
10,00005572,bruxelles boulevard anspacht galeries anspach,bruxelles,belgique,bruxelles boulevard anspacht galeries anspach ...,[],None,None,None,None,None
13,00007170,bruxelles allemagne,,,bruxelles allemagne,[],None,None,None,None,Bruxelles
15,00007202,bruxelles grand'place bruxelles,,,bruxelles grand'place,"[{'place_id': '62161128', 'licence': 'Data © O...","Grand-Place, Quartier du Centre, Bruxelles, Vi...",pedestrian,Ville de Bruxelles,Belgique,La Grand'Place de Bruxelles


In [46]:
#deuxième version de wikipedia
import requests
import functools
from ngram import NGram


def get_similar(data, target):
    G = NGram(target)
    return G.find(data)


def get_similars(data, target, threshold):
    G = NGram(target)
    return G.search(data, threshold=threshold)[0][0]


class cache(object):

    def __init__(self, fn):
        self.fn = fn
        self._cache = {}
        functools.update_wrapper(self, fn)

    def __call__(self, *args, **kwargs):
        key = str(args) + str(kwargs)
        if key in self._cache:
            ret = self._cache[key]
        else:
            ret = self._cache[key] = self.fn(*args, **kwargs)

        return ret

    def clear_cache(self):
        self._cache = {}


@cache
def gsrsearch(query, lang="FR", results=10):
    '''
    Do a Wikipedia search for `query` and 
    return a list of tuples (page title, url).
    Use ngram module to get the best match

    Keyword arguments:

    * lang : language iso code : en, fr, etc.
    * results - the maximum number of results returned

    '''

    search_params = {
        'action': 'query',
        'generator': 'search',
        'prop': 'info',
        'inprop': 'url',
        'gsrlimit': results,
        'gsrsearch': query,
        'format': 'json'
    }

    url = "https://{}.wikipedia.org/w/api.php?".format(lang.lower())

    raw_results = requests.get(url, params=search_params).json()

    try:

        search_results = [d['title']
                          for i, d in raw_results['query']['pages'].items()]
        url_results = [d['fullurl']
                       for i, d in raw_results['query']['pages'].items()]
    except KeyError:
        return None

    try:

        most_similar = get_similars(query, search_results, 0.15)
        return most_similar + " (%s)" % (url_results[search_results.index(most_similar)])

    except IndexError:

        return list(zip(search_results, url_results))

In [48]:
gb['wikipedia_improved'] = gb['loc_concat'].apply(lambda x: gsrsearch(x))

In [49]:
gb

,beeldid,spatial_value,loc_name,country_name,loc_concat,nominatim,main_loc,type_loc,city_loc,country_loc,wikipedia_test,wikipedia_improved
0,00000146,namur,namur,belgique,namur belgique,"[{'place_id': '178244718', 'licence': 'Data © ...","Namur, Wallonie, Belgique",city,Namur,Belgique,Namur,Rivière (Belgique) (https://fr.wikipedia.org/w...
1,00000429,belgique,,belgique,belgique,"[{'place_id': '179136572', 'licence': 'Data © ...",Belgique,administrative,None,Belgique,Belgique,Belgique (https://fr.wikipedia.org/wiki/Belgique)
2,00000909,ostende,,france,ostende france,"[{'place_id': '48801331', 'licence': 'Data © O...","Ostende, Rue d'Ostende, Saint-Quentin, Aisne, ...",bus_stop,None,France,Ostende,Ostende (https://fr.wikipedia.org/wiki/Ostende)
3,00004445,grand'poste bruxelles,,belgique,grand'poste bruxelles belgique,[],None,None,None,None,Verviers,"[(André et Jean Polak, https://fr.wikipedia.or..."
5,00005492,place de monnaie,bruxelles,belgique,place de monnaie bruxelles belgique,"[{'place_id': '101462620', 'licence': 'Data © ...","Place de la Monnaie, Quartier du Centre, Bruxe...",pedestrian,Ville de Bruxelles,Belgique,Place de la Monnaie (Bruxelles),Place de la Monnaie (Bruxelles) (https://fr.wi...
6,00005535,bruxelles bruxelles,bruxelles,belgique,bruxelles belgique,"[{'place_id': '17872759', 'licence': 'Data © O...","Bruxelles, Bruxelles-Capitale, 1000, Belgique",city,Bruxelles,Belgique,Communauté française de Belgique,Bruxelles (https://fr.wikipedia.org/wiki/Bruxe...
8,00005547,au petit sablon conservatoire,bruxelles,belgique,au petit sablon conservatoire bruxelles belgique,[],None,None,None,None,Ville de Bruxelles,"[(Alfred Courtens, https://fr.wikipedia.org/wi..."
10,00005572,bruxelles boulevard anspacht galeries anspach,bruxelles,belgique,bruxelles boulevard anspacht galeries anspach ...,[],None,None,None,None,None,None
13,00007170,bruxelles allemagne,,,bruxelles allemagne,[],None,None,None,None,Bruxelles,Bruxelles (https://fr.wikipedia.org/wiki/Bruxe...
15,00007202,bruxelles grand'place bruxelles,,,bruxelles grand'place,"[{'place_id': '62161128', 'licence': 'Data © O...","Grand-Place, Quartier du Centre, Bruxelles, Vi...",pedestrian,Ville de Bruxelles,Belgique,La Grand'Place de Bruxelles,La Grand'Place de Bruxelles (https://fr.wikipe...
